In [1]:
import joblib
import numpy as np

import modularised_utils as mut
import Linear_Additive_Noise_Models as lanm
import operations as ops

import params

np.random.seed(0)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/giofelekis/opt/anaconda3/envs/erica/lib/python3.12/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/giofelekis/opt/anaconda3/envs/erica/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/giofelekis/opt/anaconda3/envs/erica/lib/python3.12/site-packages/ipykernel/kernelapp.py", lin

In [2]:
experiment = 'synth1'

In [3]:
# Define the radius of the Wasserstein balls (epsilon, delta) and the size for both models.
epsilon         = params.radius[experiment][0]
ll_num_envs     = params.n_envs[experiment][0]

delta           = params.radius[experiment][1]
hl_num_envs     = params.n_envs[experiment][1]

# Define the number of samples per environment. Currently every environment has the same number of samples
num_llsamples   = params.n_samples[experiment][0]
num_hlsamples   = params.n_samples[experiment][1]

distance_err    = 'wass'

In [4]:
Dll = mut.load_samples(experiment)[None][0] 
Gll = mut.load_ll_model(experiment)[0]
Ill = mut.load_ll_model(experiment)[1]


Dhl = mut.load_samples(experiment)[None][1] 
Ghl = mut.load_hl_model(experiment)[0]
Ihl = mut.load_hl_model(experiment)[1]

omega = mut.load_omega_map(experiment)

In [5]:
ll_coeffs = mut.get_coefficients(Dll, Gll)
hl_coeffs = mut.get_coefficients(Dhl, Ghl) 

In [7]:
ll_coeffs

{('Smoking', 'Tar'): np.float64(0.2801809653886389),
 ('Tar', 'Cancer'): np.float64(0.21674035424599092)}

In [6]:
# # [Not suggested] In case we want to explore also the interventional --> worse estimation!
# Dlls, Dhls = [], []
# for dpair in list(mut.load_samples(experiment).values()):
#     Dlls.append(dpair[0])
#     Dhls.append(dpair[1])
    
# ll_coeffs = mut.get_coefficients(Dlls, Gll)
# hl_coeffs = mut.get_coefficients(Dhls, Ghl) 

In [7]:
U_ll_hat, mu_U_ll_hat, Sigma_U_ll_hat = mut.lan_abduction(Dll, Gll, ll_coeffs)
U_hl_hat, mu_U_hl_hat, Sigma_U_hl_hat = mut.lan_abduction(Dhl, Ghl, hl_coeffs)

In [8]:
LLmodels = {}
for iota in Ill:
    LLmodels[iota] = lanm.LinearAddSCM(Gll, ll_coeffs, iota)
    
HLmodels, Dhl_samples = {}, {}
for eta in Ihl:
    HLmodels[eta] = lanm.LinearAddSCM(Ghl, hl_coeffs, eta)

In [9]:
# Ambiguity set construction: Based on epsilon and delta include distribution (as many as the num_envs) that
# pass the "gelbrich" test.
ll_moments = mut.sample_moments_U(mu_hat    = mu_U_ll_hat,
                                  Sigma_hat = Sigma_U_ll_hat,
                                  bound     = epsilon,
                                  num_envs  = ll_num_envs)

A_ll       = mut.sample_distros_Gelbrich(ll_moments) #Low-level: A_epsilon


hl_moments = mut.sample_moments_U(mu_hat    = mu_U_hl_hat,
                                  Sigma_hat = Sigma_U_hl_hat,
                                  bound     = delta,
                                  num_envs  = hl_num_envs)

A_hl       = mut.sample_distros_Gelbrich(hl_moments) #High-level A_delta

In [10]:
abstraction_errors             = {}
abstraction_env_errors         = {}
max_env_avg_interv_error_value = -np.inf
max_env_avg_interv_error_key   = None

for lenv in A_ll:

    Dll_noise      = lenv.sample(num_llsamples)[0]
    ll_environment = mut.get_exogenous_distribution(Dll_noise)

    for henv in A_hl:
        Dhl_noise      = henv.sample(num_hlsamples)[0]
        hl_environment = mut.get_exogenous_distribution(Dhl_noise)

        total_ui_error = 0
        num_distros    = len(Ill)

        n, m  = len(LLmodels[None].endogenous_vars), len(HLmodels[None].endogenous_vars)

        T     = mut.sample_stoch_matrix(n, m)

        for iota in Ill:
            llcm   = LLmodels[iota]
            hlcm   = HLmodels[omega[iota]]
            llmech = llcm.compute_mechanism()
            hlmech = hlcm.compute_mechanism()
            error  = mut.ui_error_dist(distance_err, lenv, henv, llmech, hlmech, T)

            total_ui_error += error

        avg_interv_error = total_ui_error/num_distros

        if avg_interv_error > max_env_avg_interv_error_value:
            max_env_avg_interv_error_value = avg_interv_error
            max_env_avg_interv_error_key   = (lenv, henv)

        abstraction_errors[str(T)] = avg_interv_error
        abstraction_env_errors['ll: '+str(ll_environment.means_)+' hl: '+str(hl_environment.means_)] = avg_interv_error

/Users/giofelekis/opt/anaconda3/envs/erica/lib/python3.12/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [11]:
max_tau   = max(abstraction_errors, key=abstraction_errors.get)
max_error = abstraction_errors[max_tau]

print(f"Abstraction: {max_tau}, Error: {max_error}")
print('==============================================================================' )
max_lenv = max_env_avg_interv_error_key[0]
max_henv = max_env_avg_interv_error_key[1]

print(f"max LL mean vector = {max_lenv.means_}")
print(f"max LL covariance = {max_lenv.covariances_}")
print( )

print(f"max HL mean vector = {max_henv.means_}")
print(f"max HL covariance = {max_henv.covariances_}")
print('==============================================================================' )
print(f"max environment, average interventional abstraction error = {max_env_avg_interv_error_value}")

Abstraction: [[0.21761458 0.78238542]
 [0.75095088 0.24904912]
 [0.86664525 0.13335475]], Error: 1.080267499893765
max LL mean vector = [[0.03909886 0.02270429 0.149256  ]]
max LL covariance = [[[0.82603265 0.         0.        ]
  [0.         2.03002425 0.        ]
  [0.         0.         0.84745966]]]

max HL mean vector = [[ 0.07752375 -0.03284999]]
max HL covariance = [[[1.03175598 0.        ]
  [0.         0.77764308]]]
max environment, average interventional abstraction error = 1.080267499893765
